In [2]:
import numpy
import numpy as np
import pandas
import pandas as pd
from numba import njit
import sys,glob
import zipfile, os, time
import urllib3
from datetime import datetime, timedelta
import glob
import h5py
sys.path.append(os.getcwd()+'/../cds-backend/code/')
sys.path.append(os.getcwd()+'/../harvest/code/')
from harvest_convert_to_netCDF_newfixes import write_dict_h5
import cds_eua3 as eua
eua.logging_set_level(30)
import xarray as xr

import cdsapi, zipfile, os, time
#import schedule
import copy
from shutil import copyfile
import multiprocessing
import pickle

In [24]:
files = glob.glob('/raid60/scratch/uli/IGRA/*/*global_cdm-lev.csv')
print(len(files_h))
files_h

41


['/raid60/scratch/uli/IGRA_H/1978/IGRA_H_19780101_19781231_global_cdm-lev.csv',
 '/raid60/scratch/uli/IGRA_H/1979/IGRA_H_19780101_19791231_global_cdm-lev.csv',
 '/raid60/scratch/uli/IGRA_H/1980/IGRA_H_19780101_19801231_global_cdm-lev.csv',
 '/raid60/scratch/uli/IGRA_H/1981/IGRA_H_19780101_19811231_global_cdm-lev.csv',
 '/raid60/scratch/uli/IGRA_H/1982/IGRA_H_19780101_19821231_global_cdm-lev.csv',
 '/raid60/scratch/uli/IGRA_H/1983/IGRA_H_19780101_19831231_global_cdm-lev.csv',
 '/raid60/scratch/uli/IGRA_H/1984/IGRA_H_19840101_19841231_global_cdm-lev.csv',
 '/raid60/scratch/uli/IGRA_H/1985/IGRA_H_19850101_19851231_global_cdm-lev.csv',
 '/raid60/scratch/uli/IGRA_H/1986/IGRA_H_19850101_19861231_global_cdm-lev.csv',
 '/raid60/scratch/uli/IGRA_H/1987/IGRA_H_19850101_19871231_global_cdm-lev.csv',
 '/raid60/scratch/uli/IGRA_H/1988/IGRA_H_19850101_19881231_global_cdm-lev.csv',
 '/raid60/scratch/uli/IGRA_H/1989/IGRA_H_19850101_19891231_global_cdm-lev.csv',
 '/raid60/scratch/uli/IGRA_H/1990/IGRA_H

In [25]:
# stations = []
# for i in range(len(files)):
#     if i == 0:
#         chunk = pd.read_csv(files[i], header=12, index_col=False, usecols=['location_longitude', 'location_latitude', 'station_name'])
#         stations = chunk.drop_duplicates(['station_name'])
#     else:
#         chunk = pd.read_csv(files[i], header=12, index_col=False, usecols=['location_longitude', 'location_latitude', 'station_name'])
#         stations = pd.concat([stations, chunk.drop_duplicates(['station_name'])], ignore_index=True)
# stations = stations.drop_duplicates('station_name')
# pickle.dump( stations, open( "stations.p", "wb" ))

In [56]:
stations = pickle.load( open( "stations.p", "rb" ))
stations

,station_name,location_longitude,location_latitude
0,KRM00091487,-159.3670,3.8500
1,CHM00055664,87.0830,28.6330
2,CHM00056444,98.8830,28.4500
3,BMM00048062,92.8830,20.1330
4,CHM00057290,114.0500,32.9670
...,...,...,...
36592,INM00043110,73.3333,16.9833
37459,INM00042165,73.3000,28.0000
37460,INM00042328,70.9167,26.9000
37461,RPM00098233,121.7525,17.6375


In [57]:
for i in range(len(stations.station_name)):
    files = glob.glob('/raid60/scratch/uli/temp100/IGRA_' + stations.station_name.iloc[i] + '*.p')
    temp = []
    for j in files:
        temp.extend(pickle.load( open( j, "rb" )))
    selected = np.asarray(temp)[~np.isnan(np.asarray(temp))]
    datlen = len(selected)
    try:
        coefficients, residuals, _, _, _ = np.polyfit(range(datlen),selected,1,full=True)
        stations.station_name.iloc[i] = coefficients[0]
    except:
        stations.station_name.iloc[i] = np.nan

/jetfs/home/uvoggenberger/.local/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/jetfs/home/uvoggenberger/.local/lib/python3.8/site-packages/numpy/lib/polynomial.py:628: RuntimeWarning: invalid value encountered in true_divide
  lhs /= scale


In [58]:
stations

,station_name,location_longitude,location_latitude
0,0.00651639,-159.3670,3.8500
1,0.00264981,87.0830,28.6330
2,-0.0449753,98.8830,28.4500
3,0.000255236,92.8830,20.1330
4,0.0625672,114.0500,32.9670
...,...,...,...
36592,0.00392664,73.3333,16.9833
37459,-0.18,73.3000,28.0000
37460,-0.212747,70.9167,26.9000
37461,0.000229997,121.7525,17.6375


In [59]:
pickle.dump( stations, open( "IGRA_Trend.p", "wb" ))

In [50]:
np.asarray(temp)[~np.isnan(np.asarray(temp))]

array([191.85, 191.65, 192.65, 192.05, 192.85, 193.25, 192.45, 195.85,
       192.85, 194.45, 191.45, 197.65, 197.65, 194.05, 196.25, 195.45,
       198.05, 193.45, 195.05, 195.25, 194.85, 192.05, 199.05, 189.65,
       190.25, 190.85, 194.25, 191.85, 191.65, 192.65, 192.05, 192.85,
       193.25, 192.45, 195.85, 192.85, 194.45, 191.45, 197.65, 197.65,
       194.05, 196.25, 195.45, 198.05, 193.45, 195.05, 195.25, 194.85,
       192.05, 199.05, 189.65, 190.25, 190.85, 194.25, 190.65, 194.65,
       191.65, 195.65, 192.25, 197.05, 197.85, 195.25, 193.65, 195.05,
       205.85, 198.05, 197.25, 195.65, 210.05, 196.25, 193.85, 194.05,
       195.25, 200.45, 193.85, 195.25, 199.25, 200.65, 196.05, 197.85,
       195.65, 198.45, 197.25, 194.85, 196.05, 192.25, 191.85, 191.65,
       192.65, 192.05, 192.85, 193.25, 192.45, 195.85, 192.85, 194.45,
       191.45, 197.65, 197.65, 194.05, 196.25, 195.45, 198.05, 193.45,
       195.05, 195.25, 194.85, 192.05, 199.05, 189.65, 190.25, 190.85,
      

In [37]:
stations

,station_name,location_longitude,location_latitude
0,0.00651639,-159.3670,3.8500
1,CHM00055664,87.0830,28.6330
2,CHM00056444,98.8830,28.4500
3,BMM00048062,92.8830,20.1330
4,CHM00057290,114.0500,32.9670
...,...,...,...
36592,INM00043110,73.3333,16.9833
37459,INM00042165,73.3000,28.0000
37460,INM00042328,70.9167,26.9000
37461,RPM00098233,121.7525,17.6375


In [60]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

ax = plt.axes([0, 0, 1, 1], projection=ccrs.PlateCarree())
ax.add_feature(cfeature.OCEAN, zorder=0)
ax.coastlines()

da = stations
reduced = np.asarray(da['station_name'])
longitudes = list(da['location_longitude'])
latitudes = list(da['location_latitude'])
radiuses = [20]*len(da['location_latitude']) #(da['explore_tabs.sections.listings.listing.reviews_count'] / 50.0) ** 2

import matplotlib.cm as cm
colors = np.array(colors, dtype=np.float)
# col = [cm.tab20(x) for x in range(len(reduced))]
col = [cm.tab20(x/len(reduced)) for x in range(len(reduced))]

for j in range(len(reduced)):
    if(str(reduced[j]) == '-' or str(reduced[j]) == '?'):
        continue
    lat = []
    lon = []
    for i in range(len(dx.sensor_id)):
        if(str(dx.sensor_id.iloc[i])[0] == str(reduced[j])):
            lat.append(dx.iloc[i].latitude)
            lon.append(dx.iloc[i].longitude)
    ax.scatter(lon,
               lat,
               color=col[j],
               label=reduced[j],
               s=40,
               zorder=2,
               alpha=0.9)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
          fancybox=True, shadow=True, ncol=10)
plt.title('Radiosonde Types used in March 1985')
plt.savefig('radiosonde_types_used_in_march_1985')



ModuleNotFoundError: No module named 'cartopy'

In [9]:
def savetoarray(file, dataset: str='IGRA', dest: str='./temp100/'):
    yr = file.split('/')[5]
    plev = 10000
    chunk = pd.read_csv(file, header=12, index_col=False, usecols=['station_name', 'observed_value', 'air_pressure'])
    stations = chunk.station_name.drop_duplicates()
    for i in stations:
        if not os.path.isfile( dest + dataset + '_' + i + '_' + yr + '_' + str(plev) + ".p"):
            da = chunk[chunk.station_name == i]
            da = da[da.air_pressure == plev]
            pickle.dump( np.asarray(da.observed_value), open( dest + dataset + '_' + i + '_' + yr + '_' + str(plev) + ".p", "wb" ))

In [ ]:
if __name__ == '__main__':
    pool = multiprocessing.Pool(processes=20)
    result_list = pool.map(savetoarray, files[10:20])
    print(result_list)

In [ ]:
print('!')

In [38]:
lon = igra.report_timestamp.drop_duplicates()
lat = igra.location_longitude.drop_duplicates()
print(len(lon))
print(len(lat))

1304
1132


In [10]:
# IGRA and its harmonized version
for i in range(len(files)):
    print(i)
    if i == 0:
#         igra = pandas.read_csv(files[i], header=12)
        igrah = pandas.read_csv(files_h[i], header=12)
    else:
#         igra = igra.append(pandas.read_csv(files[i], header=12))
        igrah = igrah.append(pandas.read_csv(files_h[i], header=12))

0
1
2
3
4


KeyboardInterrupt: 

In [ ]:
preslist = [10000, 20000, 50000, 85000,]
for i in range(len(preslist)):
    c = comp[comp.plev == preslist[i]]
    c.time = pandas.to_datetime(c.time, utc=True)
    xax = np.asarray(c.time)
    
    selected = c.ta
    datlen = len(selected)
    coefficients, residuals, _, _, _ = np.polyfit(range(datlen),selected,1,full=True)
    mse = residuals[0]/(datlen)
    nrmse = np.sqrt(mse)/(selected.max() - selected.min())

    fig, (ax1) = plt.subplots(1, 1)
    ax1.plot(xax, np.asarray(selected), label = 'COMP', alpha=0.7)
    ax1.plot(xax, [coefficients[0]*x + coefficients[1] for x in range(len(selected))], label = 'COMP trend')
    ax1.set(xlabel ='time [multiples of 12h]', ylabel='temperature [K]', title='COMP Trend '+str(int(preslist[i]/100))+'hPa '+loc)
    textstr = 'Slope ' + str(coefficients[0]) +'\n'+'NRMSE: ' + str(nrmse)
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    ax1.text(0.03, 0.95, textstr, transform=ax1.transAxes, fontsize=16, verticalalignment='top', bbox=props)


    ax1.legend( loc='best')
    plt.show()
    plt.close(fig)